In [1]:
import os
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf

## initialize()
Initializes Hydra and add the config_path to the config search path. config_path is relative to the parent of the caller, in this case it is realtive to the directory containing this Notebook.

In [6]:
with initialize(version_base=None, config_path="data"):
    cfg = compose(config_name="config")
    print(cfg)

{'wind_system_capacity': 1500, 'solar_system_capacity': 1500, 'wind_turbine_model': 'GE 1.5sle', 'file_paths': {'power_data': './data/power_data.csv', 'wind_data': './data/wind_data_berkeley.csv', 'solar_data': './data/solar_data_berkeley.csv', 'solar_config': './data/pvwatts_config.json', 'wind_config': './data/windpower_config.json', 'wind_turbines': './data/Wind_Turbines.csv', 'carbon_data': './data/carbon_data.csv'}}


In [7]:
import math

def automatic_farm_layout(
    desired_farm_size: float, wind_turbine_kw_rating: float, wind_turbine_rotor_diameter: float
):
    num = math.floor(desired_farm_size / wind_turbine_kw_rating)
    if num <= 1:
        num = 1
    num_turbines = num

    x = [0] * num_turbines
    y = [0] * num_turbines

    # Assume they are laid out in roughly a square
    rows = math.floor(math.sqrt(num_turbines))
    cols = num_turbines / rows
    while rows * math.floor(cols) != num_turbines:  # If not evenly divisible
        rows -= 1  # Decrease the number of rows until it does divide evenly
        # If num_turbines is prime, this will continue until rows = 1
        cols = num_turbines / rows

    # Use default spacing assumptions, in multiples of rotor diameters
    spacing_x = 8 * wind_turbine_rotor_diameter
    spacing_y = 8 * wind_turbine_rotor_diameter

    # First turbine placement
    x[0] = 0
    y[0] = 0

    # Remainder of turbines
    for i in range(1, num_turbines):
        x[i] = (i - cols * math.floor(i / cols)) * spacing_x
        y[i] = math.floor(i / cols) * spacing_y

    return {
        "wind_farm_xCoordinates": x,
        "wind_farm_yCoordinates": y,
    }

In [10]:
import json
import hydra
from vessim.signal import SAMSignal
from vessim.power_meter import FilePowerMeter
from vessim.actor import ComputingSystem, Generator
from vessim.controller import Monitor
from vessim.cosim import Environment
import pandas as pd

def main(cfg):
    all_turbines = pd.read_csv(cfg.file_paths.wind_turbines, on_bad_lines="warn")
    turbine_data = all_turbines[all_turbines["Name"] == cfg.wind_turbine_model]
    turbine_rating = int(turbine_data["kW Rating"].values[0])
    turbine_rotor_diameter = int(turbine_data["Rotor Diameter"].values[0])
    turbine_power_curve = [
        float(value) for value in turbine_data["Power Curve Array"].values[0].split("|")
    ]
    turbine_wind_speeds = [
        float(value) for value in turbine_data["Wind Speed Array"].values[0].split("|")
    ]

    # Create wind config object
    with open(cfg.file_paths.wind_config, "r", errors="replace") as file:
        wind_config = json.load(file)
    farm_layout = automatic_farm_layout(
        desired_farm_size=cfg.wind_system_capacity,
        wind_turbine_kw_rating=turbine_rating,
        wind_turbine_rotor_diameter=turbine_rotor_diameter,
    )

    wind_config = {
        **wind_config,
        **farm_layout,
        "system_capacity": cfg.wind_system_capacity,
        "wind_turbine_powercurve_windspeeds": turbine_wind_speeds,
        "wind_turbine_powercurve_powerout": turbine_power_curve,
        "wind_turbine_rotor_diameter": turbine_rotor_diameter,
    }

    # Create solar config object
    with open(cfg.file_paths.solar_config, "r", errors="replace") as file:
        solar_config = json.load(file)

    solar_config["system_capacity"] = cfg.solar_system_capacity

    environment = Environment(sim_start="2020-05-01 00:00:00")

    monitor = Monitor()  # stores simulation result on each step
    environment.add_microgrid(
        actors=[
            ComputingSystem(
                power_meters=[
                    FilePowerMeter(
                        file_path=cfg.file_paths.power_data,
                        unit="MW",
                        date_format="%a %d %b %Y %H:%M:%S GMT",
                    )
                ],
                pue=1.07,
            ),
            Generator(
                signal=SAMSignal(
                    model="Windpower",
                    weather_file=cfg.file_paths.wind_data,
                    config_object=wind_config,
                ),
                name="Wind",
            ),
            Generator(
                signal=SAMSignal(
                    model="Pvwattsv8",
                    weather_file=cfg.file_paths.solar_data,
                    config_object=solar_config,
                ),
                name="Solar",
            ),
        ],
        controllers=[monitor],
        step_size=60,  # global step size (can be overridden by actors or controllers)
    )

    environment.run(until=24 * 3600 * 14)  # 14 Tage
    monitor.to_csv("result.csv")

    # Load the CSV file and calculate statistics
    df = pd.read_csv("result.csv", index_col=0, parse_dates=True)
    abs_p_delta = df["p_delta"].abs()
    avg_abs_p_delta = abs_p_delta.mean()
    std_abs_p_delta = abs_p_delta.std()

    # Calculate the embodied carbon of the wind power
    sum_wind_power_watts = df["actor_infos.Wind.p"].sum()
    total_wind_power_kWh = (sum_wind_power_watts / 1000) * (1 / 60)
    embodied_carbon_wind_grams_co2 = 12 * total_wind_power_kWh

    # Calculate the embodied carbon of the solar power
    sum_solar_power_watts = df["actor_infos.Solar.p"].sum()
    total_solar_power_kWh = (sum_solar_power_watts / 1000) * (1 / 60)
    embodied_carbon_solar_grams_co2 = 70 * total_solar_power_kWh

    embodied_carbon = embodied_carbon_wind_grams_co2 + embodied_carbon_solar_grams_co2

    # embodied_carbon = embodied_carbon_solar_grams_co2

    # Calculate the operational carbon
    carbon_data = pd.read_csv(cfg.file_paths.carbon_data, index_col=0, parse_dates=True)

    carbon_data.index = carbon_data.index.tz_localize(None)

    carbon_data_resampled = carbon_data.resample("60S").ffill()
    carbon_data_filtered = carbon_data_resampled.loc[df.index.min() : df.index.max()]

    merged_data = df.merge(carbon_data_filtered, left_index=True, right_index=True, how="left")

    merged_data["carbon_emissions"] = merged_data.apply(
        lambda row: (
            (-1 * row["p_delta"] / 1000 * (1 / 60) * row["carbon_intensity"])
            if row["p_delta"] < 0
            else 0
        ),
        axis=1,
    )

    operational_carbon = merged_data["carbon_emissions"].sum()

    merged_data.to_csv(
        hydra.core.hydra_config.HydraConfig.get().runtime.output_dir + "/merged_data.csv"
    )

    return operational_carbon, embodied_carbon, avg_abs_p_delta

print(cfg.file_paths)
main(cfg)

{'power_data': './data/power_data.csv', 'wind_data': './data/wind_data_berkeley.csv', 'solar_data': './data/solar_data_berkeley.csv', 'solar_config': './data/pvwatts_config.json', 'wind_config': './data/windpower_config.json', 'wind_turbines': './data/Wind_Turbines.csv', 'carbon_data': './data/carbon_data.csv'}


/var/folders/r9/bspfb4ms4bbgmqphvml3b5_w0000gn/T/ipykernel_63278/3687159361.py:11: ParserWarning: Skipping line 20: expected 6 fields, saw 7

  all_turbines = pd.read_csv(cfg.file_paths.wind_turbines, on_bad_lines="warn")

        ____                              _ _
       /    \                            (_) |
  ____/      \  _ __ ___   ___  ___  __ _ _| | __
 /    \      / | '_ ` _ \ / _ \/ __|/ _` | | |/ /
/      \____/  | | | | | | (_) \__ \ (_| | |   <
\      /    \  |_| |_| |_|\___/|___/\__,_|_|_|\_\
 \____/      \____
 /    \      /    \     mosaik: 3.4.0
/      \____/      \       API: 3.0.13
\      /    \      /    Python: 3.9.6
 \____/      \____/         OS: macOS-15.1.1-arm64-arm-64bit
      \      /            Docs: https://mosaik.readthedocs.io/en/3.4.0/
       \____/     Get in touch: https://github.com/orgs/OFFIS-mosaik/discussions



RuntimeError: Cannot run the event loop while another loop is running